# init

In [1]:
import os 
# Go one folder back
os.chdir('..')
from Python_scripts.data_processing import *
from Python_scripts.log_likelihood import *
from Python_scripts.utilities import *

import glob
import numpy as np
import pymc3 as pm
import theano.tensor as tt   

utility_class = utility_functions()
subjects_dataframe, subjects_dataframe_gain, subjects_dataframe_loss = data_processing_func()

/Users/danyu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/danyu/Documents/DTU/Bachelorprojekt/Bachelor_project/Python_scripts/data_processing.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ug_1['choose'][ug_1.choose != '�ܾ�'] = 1
/Users/danyu/Documents/DTU/Bachelorprojekt/Bachelor_project/Python_scripts/data_processing.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ug_1['choose'][ug_1.choose == '

# Multimodels

In [13]:

def multimodels(person, us, utility_functions_class, N_SAMPLES = 5000):
    receivor = np.array(person['receivor'])
    #proposer=10-receivor
    choice = np.array(person['choose'])
    utility_functions_class.set_responder(receivor)
    # np.isnan() can now be applied on "choice"
    choice = np.array([0 if np.isnan(x) else x for x in choice])
    #def mcmc(receivor, choice, N_SAMPLES):
    # Define the MCMC model
    with pm.Model() as model:
        alpah_1=pm.Uniform(f'alpah_1', lower = 0, upper =2)

        alpah_2=pm.Uniform(f'alpah_2', lower = 0, upper =2)
        lamda_2=pm.Uniform(f'lamda_2', lower = -1, upper =1)
       
        # Mixture model
        #us = [util.Fehr_Schmidt(alpha),util.U1(lamda1)]
        #us= [receivor - alpha * (proposer - receivor),lamda1 * proposer]
        switch=pm.Bernoulli('switch',0.5)
        print(us[0])
        print(type(np.int64(1==switch)))
        print(type((1-switch)))
        print(type(pm.math.switch(np.array(0==switch),1,1.2)))


        p = pm.Deterministic('p', 
                             pm.math.switch(pm.math.eq(0, switch),
                              utility_functions_class.P(us[0](alpah_1)),
                              utility_functions_class.P(us[1]([alpah_2,lamda_2]))))
        

        #p = pm.Deterministic('p', utility_functions_class.P(us[0](alpah_1))*switch
        #                     +utility_functions_class.P(us[1]([alpah_2,lamda_2]))*(1-switch))

        observed = pm.Bernoulli('obs', p, observed = choice)
        # Perform sampling
        #trace = pm.sample(2000, tune=1000, return_inferencedata=True)

        step = pm.Metropolis()

            # Sample from the posterior using the sampling method
        trace = pm.sample(5000, step=step)

        # Sample from the posterior using MCMC
        #trace = pm.sample(5000, tune=1000, target_accept=0.9)

    # Print the summary of the posterior distribution
    #pm.summary(trace)
    return trace



In [14]:

person = subjects_dataframe_gain[9]
#receivor = np.array(person['receivor'])
#choice = np.array(person['choose'])
#

#us = [utility_class.U2, utility_class.U1]

us=[utility_class.Fehr_Schmidt,
    utility_class.multiplication_us(utility_class.Fehr_Schmidt,utility_class.U2),
    utility_class.substraction_us(utility_class.U3,utility_class.U4),
    utility_class.substraction_us(utility_class.U2,utility_class.U4),
    utility_class.U5,
    utility_class.substraction_us(utility_class.U5,utility_class.U4),
    ]


trace_multimodels = multimodels(person, us, utility_class, N_SAMPLES = 5000)

<bound method utility_functions.Fehr_Schmidt of <Python_scripts.utilities.utility_functions object at 0x7f8f323490d0>>
<class 'numpy.int64'>
<class 'theano.tensor.var.TensorVariable'>
<class 'theano.tensor.var.TensorVariable'>


/Users/danyu/opt/anaconda3/lib/python3.8/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [switch]
>Metropolis: [lamda_2]
>Metropolis: [alpah_2]
>Metropolis: [alpah_1]
CompoundStep
>Metropolis: [switch]
>Metropolis: [lamda_2]
>Metropolis: [alpah_2]
>Metropolis: [alpah_1]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 33 seconds.
The number of effective samples is smaller than 10% for some parameters.


In [15]:
pm.summary(trace_multimodels)

Got error No model on context stack. trying to find log_likelihood in translation.
/Users/danyu/opt/anaconda3/lib/python3.8/site-packages/arviz/data/io_pymc3_3x.py:98: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
switch,0.902,0.297,0.000,1.000,0.012,0.009,567.0,567.0,1.0
alpah_1,1.034,0.562,0.016,1.870,0.010,0.007,3525.0,4088.0,1.0
alpah_2,1.298,0.333,0.807,1.877,0.007,0.005,2622.0,1277.0,1.0
lamda_2,0.571,0.347,0.118,1.000,0.011,0.008,1424.0,844.0,1.0
p[0],0.000,0.000,0.000,0.000,0.000,0.000,1363.0,1228.0,1.0
p[1],0.018,0.049,0.000,0.104,0.001,0.001,1717.0,2385.0,1.0
p[2],0.999,0.002,0.993,1.000,0.000,0.000,1217.0,563.0,1.0
p[3],0.001,0.003,0.000,0.002,0.000,0.000,1383.0,1230.0,1.0
p[4],0.962,0.079,0.783,1.000,0.003,0.002,1366.0,1095.0,1.0
p[5],0.018,0.049,0.000,0.104,0.001,0.001,1717.0,2385.0,1.0


# Simulated data test

Calculate the probabilety based on utilety and the resiver  and simulate the chose

In [49]:
rep=400
lamb=0.2

resiver=[None]*rep*5
print(len(resiver))
prop=[None]*rep*5
choise=[None]*rep*5
for i in range(rep):
    for resiv in range(1,6):
        utility_class.set_responder(resiv)
        resiver[(i*5)+resiv-1]=resiv

        # Defin the u func
        u=utility_class.U2(lamb)

        #calculate the p
        p_sim=utility_class.P(u)
        prop[(i*5)+resiv-1]=p_sim
        #Simulate from binomial distrabusing 
        choise[(i*5)+resiv-1]=np.random.binomial(n=1, p=p_sim)

resiver=np.array(resiver)      
choise = np.array([0 if np.isnan(x) else x for x in choise])
type(choise)  

2000


numpy.ndarray

In [19]:
print(resiver)

[1 2 3 ... 3 4 5]


In [50]:
utility_class.set_responder(resiver)
us = [utility_class.U2, utility_class.U1]
with pm.Model() as model:
        alpha = pm.Uniform('alpha', lower = 0, upper = 1)
        lambda1 = pm.Uniform('lamda1', lower = 0, upper = 1)
        #lambda2 = pm.Uniform('lamda2', lower = -1, upper = 1)
       
        # Mixture model
        #us = [util.Fehr_Schmidt(alpha),util.U1(lamda1)]
        #us= [receivor - alpha * (proposer - receivor),lamda1 * proposer]
        switch=pm.Bernoulli('switch',0.5)
     
        # Generalized logistic formula:  p_accept=a+((k-a)/(C+np.exp(-u*c))), where k = 1, C = 1
        p = pm.Deterministic('p', utility_class.P(us[0](alpha))*(1-switch)+utility_class.P(us[1](lambda1))*switch)

        observed = pm.Bernoulli('obs', p, observed = choise)
        # Perform sampling
        #trace = pm.sample(2000, tune=1000, return_inferencedata=True)

        step = pm.Metropolis()

        # Sample from the posterior using the sampling method
        trace = pm.sample(5000, step=step)


    # Print the summary of the posterior distribution
    #pm.summary(trace)
trace

/Users/danyu/opt/anaconda3/lib/python3.8/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [switch]
>Metropolis: [lamda1]
>Metropolis: [alpha]
CompoundStep
>Metropolis: [switch]
>Metropolis: [lamda1]
>Metropolis: [alpha]


Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 31 seconds.
/Users/danyu/opt/anaconda3/lib/python3.8/site-packages/arviz/stats/diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
The number of effective samples is smaller than 25% for some parameters.


<MultiTrace: 4 chains, 5000 iterations, 6 variables>

In [51]:
pm.summary(trace)

Got error No model on context stack. trying to find log_likelihood in translation.
/Users/danyu/opt/anaconda3/lib/python3.8/site-packages/arviz/data/io_pymc3_3x.py:98: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(
/Users/danyu/opt/anaconda3/lib/python3.8/site-packages/arviz/stats/diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
switch,0.000,0.000,0.000,0.000,0.000,0.000,20000.0,20000.0,NaN
alpha,0.205,0.007,0.192,0.218,0.000,0.000,2371.0,2218.0,1.0
lamda1,0.502,0.288,0.060,0.998,0.004,0.003,3937.0,4051.0,1.0
p[0],0.886,0.007,0.872,0.899,0.000,0.000,2371.0,2218.0,1.0
p[1],0.886,0.007,0.872,0.899,0.000,0.000,2371.0,2218.0,1.0
...,...,...,...,...,...,...,...,...,...
p[1995],0.886,0.007,0.872,0.899,0.000,0.000,2371.0,2218.0,1.0
p[1996],0.886,0.007,0.872,0.899,0.000,0.000,2371.0,2218.0,1.0
p[1997],0.886,0.007,0.872,0.899,0.000,0.000,2371.0,2218.0,1.0
p[1998],0.886,0.007,0.872,0.899,0.000,0.000,2371.0,2218.0,1.0


In [53]:
trace_fs = {}

trace_fs['alpha'] = trace.get_values('alpha', burn=1000, combine=False)
trace_fs['lamda1'] = trace.get_values('lamda1', burn=1000, combine=False)
#trace_fs['lamda2'] = trace.get_values('lamda2', burn=1000, combine=False)
trace_fs['switch'] = trace.get_values('switch', burn=1000, combine=False)


In [54]:
# save trace into npz
np.savez('MCMC_tests/trace_u2u1.npz', **trace_fs)

In [56]:
# load npz
loaded_dict = np.load('MCMC_tests/trace_u2u1.npz')

In [58]:
a = loaded_dict['alpha']
a.mean()

0.20535143560989536